# 🚀 Sistema de recomendación de juegos

En este notebook se hace la experimentación para encontrar dos modelos de recomendación, que generen una lista de 5 juegos ya sea ingresando el nombre de un juego o el id de un usuario.

En el primer caso, el modelo tiene una relación ítem-ítem, esto es se toma un item, en base a que tan similar es ese ítem al resto, se recomiendan similares. En el segundo caso, el sistema de recomendación aplica el filtro user-item, esto es tomar un usuario, se encuentran usuarios similares y se recomiendan ítems que a esos usuarios similares les gustaron.

#### 📥Importaciones 

In [15]:
import pandas as pd
import numpy as np
import scipy as sp
import operator
from sklearn.metrics.pairwise import cosine_similarity
import pyarrow as pa
import pyarrow.parquet as pq

#### 📦 Se extraen los datos que se prepararon luego del EDA y se convierten en dataframe para ser utilizados por el modelo.

In [16]:
df = pd.read_csv('data/df_recomendacion.csv')
df

,user_id,item_name,rating
0,76561197970982479,Killing Floor,3
1,EndAtHallow,Killing Floor,3
2,76561198077432581,Killing Floor,3
3,76561198057958244,Killing Floor,1
4,46366536564574576346346546,Killing Floor,3
...,...,...,...
44200,ButtBurger2,Cities in Motion,1
44201,76561198064526566,Pesadelo - Regressão,2
44202,haungaraho,Trials 2: Second Edition,5
44203,UnseenPrecision,Bridge Project,3


Se un dataframe que contiene los 'user_id' como indices, los juegos ('item_name') como columnas y los ´rating' como valores.

In [17]:
piv = df.pivot_table(index=['user_id'], columns=['item_name'], values='rating')
piv

item_name,! That Bastard Is Trying To Steal Our Gold !,0RBITALIS,"10,000,000",100% Orange Juice,1001 Spikes,12 Labours of Hercules,12 Labours of Hercules II: The Cretan Bull,12 is Better Than 6,140,16bit Trader,...,inMomentum,klocki,liteCam Game: 100 FPS Game Capture,oO,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,sZone-Online,the static speaks my name,theHunter,theHunter: Primal
user_id,,,,,,,,,,,,,,,,,,,,,
--000--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ace--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ionex--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-2SV-vuLB-Kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-Azsael-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zuzuga2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zv_odd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zvanik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Se normalizan los valores del dataframe `piv` para restar la media de las calificaciones de un usuario y despues dividir por la diferencia entre el valor máximo y mínimo de las calificaciones. Esto ajusta las calificaciones de un usuario de manera que estén centradas en cero y escaladas en función de su variabilidad. Los usuarios que solo dieron una calificación o calificaron todos los juegos de la misma manera van a ser eliminados durante este proceso de normalización. Esto se debe a que estos usuarios no aportan información útil para el modelo de recomendación si todas sus calificaciones son iguales o si solo tienen una calificación.

In [18]:
# Normalización del dataframe 'piv'
piv_norm = piv.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
# Se borran las columnas que contienen solo cero o no tienen rating, se rellenan los vacios con 0 y se hace la transpuesta
piv_norm.fillna(0, inplace=True)
piv_norm = piv_norm.T
piv_norm = piv_norm.loc[:, (piv_norm != 0).any(axis=0)]
piv_norm

user_id,--000--,-Beave-,-I_AM_EPIC-,-SEVEN-,-Thyme-,-kainey9777,00000000000000000001227,00690069006900,03092002,04061993,...,zombi_anon,zomgCoBfAce,zoom-the-flash,zoozles,zourock,zsharoarkbr,zuzuga2003,zvanik,zwanzigdrei,zzoptimuszz
item_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0RBITALIS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"10,000,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100% Orange Juice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001 Spikes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
resident evil 4 / biohazard 4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sZone-Online,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
the static speaks my name,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


A los datos de esta matriz normalizada se los convierte a un formato de matriz dispersa (sparse matrix) para reducir la memoria utilizada y mejorar la eficiencia en el manejo de grandes conjuntos de datos, especialmente cuando la mayoría de los valores en la matriz son ceros. 

In [19]:
piv_sparse = sp.sparse.csr_matrix(piv_norm.values)
piv_sparse

<2813x6964 sparse matrix of type '<class 'numpy.float64'>'
	with 25041 stored elements in Compressed Sparse Row format>

Se crean dos matrices de similitud utilizando la similitud del coseno para medir la similitud entre los juegos (item_similarity) y entre los usuarios (user_similarity).

La similitud del coseno es una medida comúnmente utilizada para evaluar la similitud entre dos vectores en un espacio multidimensional. En el contexto de sistemas de recomendación y análisis de datos, la similitud del coseno se utiliza para determinar cuán similares son dos conjuntos de datos o elementos, y se calcula utilizando el coseno del ángulo entre los vectores que representan esos datos o elementos.

In [20]:
item_similarity = cosine_similarity(piv_sparse)
user_similarity = cosine_similarity(piv_sparse.T)

Se insertan las matrices anteriores en un Dataframe para estructurar y organizar los resultados de manera más accesible y comprensible.

In [21]:
#item similarity dataframe
item_sim_df = pd.DataFrame(item_similarity, index = piv_norm.index, columns = piv_norm.index)
#user similarity dataframe
user_sim_df = pd.DataFrame(user_similarity, index = piv_norm.columns, columns = piv_norm.columns)

## 🚀 Función de recomendación según un juego

Se realiza una función que devuelve una recomendación de 5 juegos en función de un juego dado, teniendo en cuenta los valores más altos de similitud del coseno. 

In [22]:
def recomendacion_juego(game):
    count = 1
    print('Similar games to {} include:\n'.format(game))
    for item in item_sim_df.sort_values(by = game, ascending = False).index[1:6]:
        print('No. {}: {}'.format(count, item))
        count +=1  

Se prueba la función 'recomendacion_juego'

In [23]:
recomendacion_juego('Killing Floor')

Similar games to Killing Floor include:

No. 1: S.T.A.L.K.E.R.: Shadow of Chernobyl
No. 2: Unreal Gold
No. 3: Rochard
No. 4: Recettear: An Item Shop's Tale
No. 5: AaAaAA!!! - A Reckless Disregard for Gravity


In [24]:
recomendacion_juego('theHunter')

Similar games to theHunter include:

No. 1: Hover : Revolt Of Gamers
No. 2: Bloons TD Battles
No. 3: Damned
No. 4: Red Faction: Guerrilla Steam Edition
No. 5: City of Steam: Arkadia


## 🚀 Función de recomendación según un usuario

Se realiza una función que genera una lista de 5 juegos recomendados para un usuario en función de las calificaciones de usuarios similares. Los juegos que son más frecuentemente recomendados por usuarios similares se consideran como las principales recomendaciones para ese usuario.

In [25]:
def recomendacion_usuario(user):
    
    # Se verifica si el usuario está presente en las columnas de piv_norm (si no está, devuelve un mensaje)
    if user not in piv_norm.columns:
        return('No data available on user {}'.format(user))
    
    # Se obtienen los usuarios más similares al usuario dado
    sim_users = user_sim_df.sort_values(by=user, ascending=False).index[1:11]
    
    best = []  # Lista para almacenar los juegos mejor calificados por usuarios similares
    most_common = {}  # Diccionario para contar cuántas veces se recomienda cada juego
    
    # Para cada usuario similar, se encuentra el juego mejor calificado y se agrega a la lista 'best'
    for i in sim_users:
        max_score = piv_norm.loc[:, i].max()
        best.append(piv_norm[piv_norm.loc[:, i]==max_score].index.tolist())
    
    # Se cuenta cuántas veces se recomienda cada juego
    for i in range(len(best)):
        for j in best[i]:
            if j in most_common:
                most_common[j] += 1
            else:
                most_common[j] = 1
    
    # Se ordenan los juegos por la frecuencia de recomendación en orden descendente
    sorted_list = sorted(most_common.items(), key=operator.itemgetter(1), reverse=True)
    
    # Se devuelven los 5 juegos más recomendados
    return sorted_list[:5]  

In [26]:
# ejemplo
recomendacion_usuario('zvanik')

[('Call of Duty: World at War', 4),
 ("Garry's Mod", 2),
 ('Counter-Strike', 1),
 ('Counter-Strike: Global Offensive', 1),
 ('Hotline Miami', 1)]

In [27]:
# ejemplo
recomendacion_usuario('76561197970982479')

[('DayZ', 2),
 ('Hotline Miami', 1),
 ('Monaco', 1),
 ('Skullgirls', 1),
 ('Valkyria Chronicles™', 1)]

## 📥 Carga de las matrices para recomendar en la API

Se guardan en formato parquet lo que permite una compresión y codificación eficiente.

In [28]:
pq.write_table(pa.Table.from_pandas(piv_norm), 'data/parquet/piv_norm.parquet')
pq.write_table(pa.Table.from_pandas(user_sim_df), 'data/parquet/user_sim_df.parquet')
pq.write_table(pa.Table.from_pandas(item_sim_df), 'data/parquet/item_sim_df.parquet')
print('Se guardaron correctamente')

Se guardaron correctamente
